# Guardian - FROM 01.01.2022


## Import Packages

In [8]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [9]:
iteration = 1

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": 
        """headline,
        firstPublicationDate,
        productionOffice, 
        bodyText, 
        charCount, 
        commentable, 
        starRating, 
        isLive"""
    ,
    "show-elements": "all",
    "show-tags": ["all"],
    "show-rights": "syndicatable",
    "from-date": "2022-01-01",
    #"to-date": "2022-06-30",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY"),
}


In [10]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [11]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [12]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=headline%2C%0A++++++++firstPublicationDate%2C%0A++++++++productionOffice%2C+%0A++++++++bodyText%2C+%0A++++++++charCount%2C+%0A++++++++commentable%2C+%0A++++++++starRating%2C+%0A++++++++isLive&show-elements=all&show-tags=all&show-rights=syndicatable&from-date=2022-01-01&order-by=newest&api-key=2d2caab5-0315-41a3-aab9-7390fade2f60&page=1
status ok
total 33281
startIndex 1
pageSize 50
pages 666
orderBy newest
---- RUNTIME STATUS ---- 
(1/666) in 0:00:00.606515s
(2/666) in 0:00:00.386738s
(3/666) in 0:00:00.596885s
(4/666) in 0:00:00.668527s
(5/666) in 0:00:00.454507s
(6/666) in 0:00:00.435585s
(7/666) in 0:00:00.595131s
(8/666) in 0:00:00.399494s
(9/666) in 0:00:00.328738s
(10/666) in 0:00:00.653901s
(11/666) in 0:00:00.375839s
(12/666) in 0:00:00.751581s
(13/666) in 0:00:00.638060s
(14/666) in 0:00:00.404799s
(15/666) in 0:00:00.620440s
(16/666) in 0:00:00.392056s
(17/666) in 0:00:00.61

In [13]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields", "rights"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [14]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33282, 14) took 0:00:00.009472s

unfold_columns:
 shape=(33282, 17) took 0:00:00.647091s

create_csv:
 shape=(33282, 17) took 0:00:04.193936s

